In [7]:
from osgeo import gdal
import numpy as np

# Input and output file paths
input_file = "/p/lustre2/lazin1/Annual_NLCD_LndCov_2023_CU_C1V0.tif"  # Replace with your NLCD GeoTIFF
output_file = "/p/lustre2/lazin1/reclassified_Annual_NLCD_LndCov_2023_CU_C1V0.tif"

# Open the NLCD GeoTIFF
raster = gdal.Open(input_file)
band = raster.GetRasterBand(1)  # Access the first raster band
data = band.ReadAsArray()

# Reclassification: Example - Urban areas as 1, others as 0
# Adjust the values to match the NLCD classes of interest
# reclassified_data = np.where((data == 21) | (data == 22) | (data == 23) | (data == 24), 1, 0)
reclassified_data = np.where(data == 11, 1, 0)
reclassified_data= reclassified_data.astype(np.uint8)

# Save the reclassified raster as a GeoTIFF
driver = gdal.GetDriverByName("GTiff")
out_raster = driver.Create(output_file, raster.RasterXSize, raster.RasterYSize, 1, gdal.GDT_Byte) #gdal.GDT_Byte)
out_band = out_raster.GetRasterBand(1)

# Write reclassified data to the output raster
out_band.WriteArray(reclassified_data)

# Copy spatial metadata (projection, geotransform) from the original raster
out_raster.SetGeoTransform(raster.GetGeoTransform())
out_raster.SetProjection(raster.GetProjection())
out_band.FlushCache()

print(f"Reclassified raster saved to {output_file}")


MemoryError: Unable to allocate 125. GiB for an array with shape (105000, 160000) and data type int64

In [3]:
import os
os.path.exists("/p/lustre2/lazin1/Annual_NLCD_LndCov_2023_CU_C1V0.tif")

True

In [14]:
np.sum(data==11)

460771999

In [5]:
# Save the reclassified raster as a GeoTIFF
driver = gdal.GetDriverByName("GTiff")
out_raster = driver.Create(output_file, raster.RasterXSize, raster.RasterYSize, 1, gdal.GDT_Byte) #gdal.GDT_Byte)
out_band = out_raster.GetRasterBand(1)

# Write reclassified data to the output raster
out_band.WriteArray(reclassified_data)

# Copy spatial metadata (projection, geotransform) from the original raster
out_raster.SetGeoTransform(raster.GetGeoTransform())
out_raster.SetProjection(raster.GetProjection())
out_band.FlushCache()

print(f"Reclassified raster saved to {output_file}")

Reclassified raster saved to /p/lustre2/lazin1/reclassified_Annual_NLCD_LndCov_2023_CU_C1V0.tif


In [3]:
data.dtype

dtype('uint8')

In [8]:
reclassified_data.astype(np.uint8)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [1]:
import os
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio import warp
# from netCDF4 import Dataset
from osgeo import gdal
from datetime import datetime, timedelta
import pandas as pd
import glob
reference_raster_files = glob.glob("/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_Flood_Images_No_Threshold_test/*.tif") # /p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_Flood_Images_No_Threshold_test #/p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_Flood_Images_No_Threshold_test_Mississippi_20190617_3AC6
#
# /p/lustre1/lazin1/RAPID_Archive_Flood_Maps/Tile_No_Threshold_Mississippi_20190617_5E5F_non_flood

output_geotiff_dir = '/p/lustre2/lazin1/cropped_LC_Harvey_20170829_D734' #Mississippi_20190617_5E5F_non_flood' #Harvey_D374'
os.makedirs(output_geotiff_dir,exist_ok=True)

# reference_raster_list = os.listdir(reference_raster_dir)





temp_raster_path = f"/p/lustre2/lazin1/Annual_NLCD_LndCov_2023_CU_C1V0.tif"
date_format = "%Y%m%d"
# Get reference raster info (extent and resolution)
for reference_raster_file in reference_raster_files:
    
    # temp_raster_path = os.path.join(output_geotiff_dir, reference_raster_file.split("/")[-1].split("crop")[0][:-1] + '.tif')
    output_geotiff = os.path.join(output_geotiff_dir, 'LC_' + reference_raster_file.split("/")[-1])
    
    
    # temp_raster_path = os.path.join(output_geotiff_dir, reference_raster_file[1].split("crop")[0][:-1]+'.tif')
    # output_geotiff = os.path.join(output_geotiff_dir, '1day_prec_' + reference_raster_file[1])
    reference_raster = reference_raster_file #os.path.join(reference_raster_dir,reference_raster_file[1])
    with rasterio.open(reference_raster) as ref:
        ref_transform = ref.transform
        ref_crs = ref.crs
        ref_shape = (ref.height, ref.width)
        ref_bounds = ref.bounds

    # with rasterio.open(
    #     temp_raster_path,
    #     "w",
    #     driver="GTiff",
    #     height=precip_1day.shape[0],
    #     width=precip_1day.shape[1],
    #     count=1,
    #     dtype="float32",
    #     crs="EPSG:4326",
    #     transform=from_origin(longitudes.min(), latitudes.max(), longitudes[1] - longitudes[0], latitudes[1] - latitudes[0])
    # ) as temp_dst:
    #     temp_dst.write((np.flip(precip_1day, axis=0)), 1)

    # Reproject and resample to match the reference raster
    with rasterio.open(temp_raster_path) as src:
        with rasterio.open(
            output_geotiff,
            "w",
            driver="GTiff",
            height=ref_shape[0],
            width=ref_shape[1],
            count=1,
            dtype="float32",
            crs=ref_crs,
            transform=ref_transform,
            nodata=-9999
        ) as dst:
            # Resample and reproject the data
            warp.reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=warp.Resampling.nearest
            )
